TODO
[x] DisbursementDateを日付に変更する
[x] RevLineCr / LowDoc OneHot (一旦Label Encoding)
[x] City / State Label Encoding
[x] DisbursementGross / GrAppv / SBA_Appv　を数値に変換
[ ] Find the difference between Urban / Not Urban
[ ] Get the population for each state 
[ ] Urban / Not Urbanのものの差を検証


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config
from sklearn.inspection import permutation_importance
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.metrics import roc_auc_score, roc_curve, make_scorer, f1_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.preprocessing import FunctionTransformer, StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

sns.set_theme(style = 'white', palette = 'viridis')
pal = sns.color_palette('viridis')

pd.set_option('display.max_rows', 100)
set_config(transform_output = 'pandas')
pd.options.mode.chained_assignment = None

In [2]:
train = pd.read_csv(r'../data/train.csv')
test = pd.read_csv(r'../data/test.csv')

In [3]:
def describe_satatistics(input_df):
    desc = pd.DataFrame(index=list(input_df))
    desc["nunique"] = train.nunique()
    desc["unique%"] = train.nunique() / len(input_df)
    desc["null"] = train.isnull().sum()
    desc["type"] = train.dtypes
    desc = pd.concat([desc,input_df.describe().T],axis=1)
    return desc

# Preprocessing

In [4]:
train["train_data"]=1
test["train_data"]=0
main_df = pd.concat([train,test])
main_df.reset_index(inplace=True,drop=True)

In [5]:
pd.set_option('display.max_columns', None)
main_df.head()

,ID,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,MIS_Status,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural,train_data
0,0,163,21,1.0,0,0,1,N,N,31-Jan-98,1.0,0,22-Sep-06,2006,PHOENIX,AZ,SD,"$80,000.00","$80,000.00","$68,000.00",0,1
1,1,84,6,1.0,4,0,0,0,N,31-Oct-93,1.0,62,30-Jun-92,1992,MCALESTER,OK,OK,"$287,000.00","$287,000.00","$229,600.00",0,1
2,2,242,45,1.0,4,90,0,N,N,31-Aug-01,1.0,42,18-Apr-01,2001,HAWTHORNE,NJ,NJ,"$31,983.00","$30,000.00","$15,000.00",1,1
3,3,237,4,1.0,0,0,0,N,N,31-Aug-07,1.0,33,6-Oct-03,2004,NASHVILLE,TN,SD,"$229,000.00","$229,000.00","$229,000.00",0,1
4,4,184,0,1.0,0,0,0,N,N,8-Jun-83,1.0,0,17-Dec-99,2000,POMONA,CA,CA,"$525,000.00","$525,000.00","$393,750.00",0,1


In [6]:
eda_df = describe_satatistics(main_df)

In [7]:
eda_df

,nunique,unique%,null,type,count,mean,std,min,25%,50%,75%,max
ID,42307,0.499994,0,int64,84615.0,42307.000000,24426.390851,0.0,21153.5,42307.0,63460.5,84614.0
Term,228,0.002695,0,int64,84615.0,108.558447,84.731943,0.0,56.0,82.0,168.0,360.0
NoEmp,196,0.002316,0,int64,84615.0,9.699510,17.365667,0.0,2.0,4.0,12.0,208.0
NewExist,2,0.000024,0,float64,84615.0,1.209632,0.407048,1.0,1.0,1.0,1.0,2.0
CreateJob,49,0.000579,0,int64,84615.0,2.185144,5.112835,0.0,0.0,0.0,2.0,86.0
RetainedJob,83,0.000981,0,int64,84615.0,3.488897,8.124039,0.0,0.0,0.0,4.0,175.0
FranchiseCode,271,0.003203,0,int64,84615.0,1990.058843,10587.667836,0.0,0.0,0.0,1.0,90709.0
RevLineCr,4,0.000047,1079,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LowDoc,6,0.000071,531,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DisbursementDate,916,0.010826,150,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
main_df["DisbursementDate"] = pd.to_datetime(main_df["DisbursementDate"])
main_df["ApprovalDate"] = pd.to_datetime(main_df["ApprovalDate"])

C:\Users\kaminogo.kenji\AppData\Local\Temp\ipykernel_14292\82921768.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  main_df["DisbursementDate"] = pd.to_datetime(main_df["DisbursementDate"])
C:\Users\kaminogo.kenji\AppData\Local\Temp\ipykernel_14292\82921768.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  main_df["ApprovalDate"] = pd.to_datetime(main_df["ApprovalDate"])


In [9]:
le = LabelEncoder()
columns_list = ["RevLineCr","LowDoc","City","State","BankState"]

for col in columns_list:
    le = LabelEncoder()
    le.fit(main_df[col])
    main_df[f"{col}_label"] = le.transform(main_df[col])


In [10]:
money_columns = ["DisbursementGross","GrAppv","SBA_Appv"]

In [11]:
for col in money_columns:
    main_df[col] = main_df[col].str.replace('$', '').replace(',', '')
    main_df[col] = main_df[col].str.replace(",","")
    main_df[col] = main_df[col].str.extract(r"(\d+)")
    main_df[col] = main_df[col].astype(int)

In [12]:
main_df[money_columns].dtypes

DisbursementGross    int32
GrAppv               int32
SBA_Appv             int32
dtype: object

In [13]:
train

,ID,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,MIS_Status,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural,train_data
0,0,163,21,1.0,0,0,1,N,N,31-Jan-98,1,0,22-Sep-06,2006,PHOENIX,AZ,SD,"$80,000.00","$80,000.00","$68,000.00",0,1
1,1,84,6,1.0,4,0,0,0,N,31-Oct-93,1,62,30-Jun-92,1992,MCALESTER,OK,OK,"$287,000.00","$287,000.00","$229,600.00",0,1
2,2,242,45,1.0,4,90,0,N,N,31-Aug-01,1,42,18-Apr-01,2001,HAWTHORNE,NJ,NJ,"$31,983.00","$30,000.00","$15,000.00",1,1
3,3,237,4,1.0,0,0,0,N,N,31-Aug-07,1,33,6-Oct-03,2004,NASHVILLE,TN,SD,"$229,000.00","$229,000.00","$229,000.00",0,1
4,4,184,0,1.0,0,0,0,N,N,8-Jun-83,1,0,17-Dec-99,2000,POMONA,CA,CA,"$525,000.00","$525,000.00","$393,750.00",0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42302,42302,283,14,1.0,0,0,1,N,N,31-Jan-98,1,0,2-Mar-95,1995,PHILADELPHIA,PA,PA,"$80,000.00","$80,000.00","$68,000.00",0,1
42303,42303,53,2,1.0,0,0,0,Y,N,3-Apr-91,1,42,6-Jun-07,2007,LOS ANGELES,CA,SD,"$5,000.00","$5,000.00","$4,250.00",1,1
42304,42304,59,6,2.0,0,0,1,N,N,28-Feb-03,1,42,14-Mar-03,2003,COLUMBUS,OH,OH,"$60,000.00","$60,000.00","$51,000.00",0,1
42305,42305,295,18,1.0,0,8,0,N,N,10-Dec-97,1,42,23-Aug-89,1989,CLOQUET,MN,MN,"$294,000.00","$294,000.00","$220,500.00",0,1


In [14]:
def adverarial_validation(label = "train and test adversarial validation"):
    adv=main_df.drop(["MIS_Status","DisbursementDate","ApprovalDate"],axis=1)
    adv_shuffle = adv.sample(frac=1)
    adv_x = adv_shuffle.drop("train_data",axis=1)
    adv_y = adv_shuffle["train_data"]
    val_scores = []
    predictions = np.zeros(len(adv))
    
    sfk = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
    cv = sfk.split(adv_x,adv_y)
    
    for fold,(idx_train,idx_valid) in enumerate(cv):
        adv_lr = LGBMClassifier(random_state=42)
        adv_lr.fit(adv_x.iloc[idx_train],adv_y.iloc[idx_train])
        val_preds = adv_lr.predict_proba(adv_x.iloc[idx_valid])[:,1]
        predictions[idx_valid] = val_preds
        val_score = roc_auc_score(adv_y.iloc[idx_valid], val_preds)
        val_scores.append(val_score)
    
    fpr, tpr, _ = roc_curve(adv['train_data'], predictions)
    
    plt.figure(figsize = (10, 10), dpi = 300)
    sns.lineplot(x=[0, 1], y=[0, 1], linestyle="--", label="Indistinguishable Datasets")
    sns.lineplot(x=fpr, y=tpr, label="Adversarial Validation Classifier")
    plt.title(f'{label} Validation = {np.mean(val_scores):.5f}', weight = 'bold', size = 17)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.show()
    

In [15]:
main_df.dtypes

ID                            int64
Term                          int64
NoEmp                         int64
NewExist                    float64
CreateJob                     int64
RetainedJob                   int64
FranchiseCode                 int64
RevLineCr                    object
LowDoc                       object
DisbursementDate     datetime64[ns]
MIS_Status                  float64
Sector                        int64
ApprovalDate         datetime64[ns]
ApprovalFY                    int64
City                         object
State                        object
BankState                    object
DisbursementGross             int32
GrAppv                        int32
SBA_Appv                      int32
UrbanRural                    int64
train_data                    int64
RevLineCr_label               int32
LowDoc_label                  int32
City_label                    int32
State_label                   int32
BankState_label               int32
dtype: object

In [16]:
# adverarial_validation()

In [17]:
main_df

,ID,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,MIS_Status,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural,train_data,RevLineCr_label,LowDoc_label,City_label,State_label,BankState_label
0,0,163,21,1.0,0,0,1,N,N,1998-01-31,1.0,0,2006-09-22,2006,PHOENIX,AZ,SD,80000,80000,68000,0,1,1,3,2208,3,42
1,1,84,6,1.0,4,0,0,0,N,1993-10-31,1.0,62,1992-06-30,1992,MCALESTER,OK,OK,287000,287000,229600,0,1,0,3,1723,36,36
2,2,242,45,1.0,4,90,0,N,N,2001-08-31,1.0,42,2001-04-18,2001,HAWTHORNE,NJ,NJ,31983,30000,15000,1,1,1,3,1214,31,31
3,3,237,4,1.0,0,0,0,N,N,2007-08-31,1.0,33,2003-10-06,2004,NASHVILLE,TN,SD,229000,229000,229000,0,1,1,3,1906,42,42
4,4,184,0,1.0,0,0,0,N,N,1983-06-08,1.0,0,1999-12-17,2000,POMONA,CA,CA,525000,525000,393750,0,1,1,3,2246,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84610,84610,243,10,1.0,3,14,0,N,N,2012-12-01,NaN,42,2012-04-23,2012,FT. WRIGHT,KY,OH,390000,150000,127500,0,0,1,3,1022,17,35
84611,84611,178,0,2.0,0,0,1,N,N,2003-11-30,NaN,0,2006-10-27,2007,PHILADELPHIA,PA,RI,100000,100000,90000,0,0,1,3,2207,38,40
84612,84612,42,1,2.0,3,9,0,Y,N,2009-02-28,NaN,33,1989-09-21,1989,ELMHURST,IL,IL,17000,17000,13600,0,0,3,3,858,14,14
84613,84613,76,15,1.0,0,0,0,N,N,2008-01-31,NaN,0,2006-04-03,2006,NASHVILLE,TN,TN,7500,7500,6375,0,0,1,3,1906,42,43


In [18]:
main_df["City"].value_counts()

City
HOUSTON           2378
PITTSBURGH        1864
SALT LAKE CITY    1269
NASHVILLE         1230
POMONA            1207
                  ... 
SALFORD              1
South Elgin          1
GLENNS FERRY         1
ATLANTIC             1
REPUBLIC             1
Name: count, Length: 3211, dtype: int64

In [19]:
tmp_df = main_df.copy()
tmp_df["main_key"] = tmp_df["State"].str.cat(tmp_df["Sector"].astype(str), sep="_")
count_df =pd.DataFrame(tmp_df["main_key"].value_counts())
count_df.reset_index(inplace = True)
top_key_list = count_df["main_key"].head(100)

In [20]:
main_df["main_key"] = main_df["State"].str.cat(main_df["Sector"].astype(str), sep="_")
main_df["main_key"] = np.where(main_df["main_key"].isin(top_key_list),main_df["main_key"],"XX")
main_df["main_key"].value_counts()

main_key
XX       32231
TX_0      3408
CA_0      3313
CA_42     2292
PA_0      2170
         ...  
FL_22      203
WA_23      203
GA_0       198
TX_53      197
CO_44      196
Name: count, Length: 101, dtype: int64

In [21]:
population_df= pd.read_csv("../data/us_popluation.csv")

In [28]:
eda_df = main_df[["State","UrbanRural","City"]].copy()

In [27]:
num1 = eda_df[eda_df["UrbanRural"]==0]["State"].nunique()
num2 = eda_df[eda_df["UrbanRural"]==1]["State"].nunique()
num3 = eda_df[eda_df["UrbanRural"]==2]["State"].nunique()
print(print(num1,num2,num3))
num1 = eda_df[eda_df["UrbanRural"]==0]["State"].nunique()
num2 = eda_df[eda_df["UrbanRural"]==1]["State"].nunique()
num3 = eda_df[eda_df["UrbanRural"]==2]["State"].nunique()
print(print(num1,num2,num3))


51 51 51
None


In [38]:
for a in range(3):
    print(eda_df[eda_df["UrbanRural"]==(a)]["City"].nunique())

2895
1979
1219


In [41]:
# eda_df02 = eda_df[[eda_df["UrbanRural"]==0]["State"].isin([eda_df["UrbanRural"]==1]["State"].unique())]
city_unique = eda_df[eda_df["UrbanRural"]==1]["State"].unique()
eda_df02 = eda_df[eda_df["UrbanRural"]==0]
eda_df02 = eda_df02[eda_df02["City"].isin(city_unique)]


In [42]:
eda_df02

,State,UrbanRural,City


In [58]:
def create_groupby_features(target_col,group_feature):
    main_df[f"{group_feature}_group_med"] = main_df.groupby(f"{group_feature}")[target_col]
    main_df[f"{group_feature}_main_key_group_diff"] = main_df[target_col] - main_df[f"{group_feature}_group_med"]

In [91]:
population_df = pd.read_csv("../data/population_for_each_state.csv")

In [92]:
population_df_02 = pd.read_csv("../data/US_population_02.csv")

In [93]:
state_df = pd.read_csv("../data/State.csv")
state_df.rename(columns={"name":"State"},inplace=True)

In [94]:
population_df = pd.merge(population_df,state_df,how ="left", on = "State")

In [96]:
population_df = pd.merge(population_df, population_df_02, how="left",on="State")

In [98]:
population_df.head()

,State,2024,per_2024,2023,2010,name2,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2003
0,California,"38,889,770",11.58%,"38,965,193","37,319,550",CA,"1,485,053","2,377,549","3,426,861","5,677,251","6,907,387","10,586,223","15,717,204","19,953,134","23,667,902","29,760,021","33,871,648","35,484,453"
1,Texas,"30,976,754",9.22%,"30,503,301","25,241,897",TX,"3,048,710","3,896,542","4,663,228","5,824,715","6,414,824","7,711,194","9,579,677","11,196,730","14,229,191","16,986,510","20,851,820","22,118,509"
2,Florida,"22,975,931",6.84%,"22,610,726","18,846,143",FL,"528,542","752,619","968,470","1,468,211","1,897,414","2,771,305","4,951,560","6,789,443","9,746,324","12,937,926","15,982,378","17,019,068"
3,New York,"19,469,232",5.80%,"19,571,216","19,399,956",NY,"7,268,894","9,113,614","10,385,227","12,588,066","13,479,142","14,830,192","16,782,304","18,236,967","17,558,072","17,990,455","18,976,457","19,190,115"
4,Pennsylvania,"12,951,275",3.86%,"12,961,683","12,711,406",PA,"6,302,115","7,665,111","8,720,017","9,631,350","9,900,180","10,498,012","11,319,366","11,793,909","11,863,895","11,881,643","12,281,054","12,365,455"


In [99]:
population_df.drop("State",axis=1,inplace=True)

In [102]:
population_df.rename(columns={"name2":"State"},inplace=True)

,2024,per_2024,2023,2010,State,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2003
0,"38,889,770",11.58%,"38,965,193","37,319,550",CA,"1,485,053","2,377,549","3,426,861","5,677,251","6,907,387","10,586,223","15,717,204","19,953,134","23,667,902","29,760,021","33,871,648","35,484,453"
1,"30,976,754",9.22%,"30,503,301","25,241,897",TX,"3,048,710","3,896,542","4,663,228","5,824,715","6,414,824","7,711,194","9,579,677","11,196,730","14,229,191","16,986,510","20,851,820","22,118,509"
2,"22,975,931",6.84%,"22,610,726","18,846,143",FL,"528,542","752,619","968,470","1,468,211","1,897,414","2,771,305","4,951,560","6,789,443","9,746,324","12,937,926","15,982,378","17,019,068"
3,"19,469,232",5.80%,"19,571,216","19,399,956",NY,"7,268,894","9,113,614","10,385,227","12,588,066","13,479,142","14,830,192","16,782,304","18,236,967","17,558,072","17,990,455","18,976,457","19,190,115"
4,"12,951,275",3.86%,"12,961,683","12,711,406",PA,"6,302,115","7,665,111","8,720,017","9,631,350","9,900,180","10,498,012","11,319,366","11,793,909","11,863,895","11,881,643","12,281,054","12,365,455"
5,"12,516,863",3.73%,"12,549,689","12,840,545",IL,"4,821,550","5,638,591","6,485,280","7,630,654","7,897,241","8,712,176","10,081,158","11,113,976","11,426,518","11,430,602","12,419,293","12,653,544"
6,"11,812,173",3.52%,"11,785,935","11,539,449",OH,"4,157,545","4,767,121","5,759,394","6,646,697","6,907,612","7,946,627","9,706,397","10,652,017","10,797,630","10,847,115","11,353,140","11,435,798"
7,"11,145,304",3.32%,"11,029,227","9,712,209",GA,"2,216,331","2,609,121","2,895,832","2,908,506","3,123,723","3,444,578","3,943,116","4,589,575","5,463,105","6,478,216","8,186,453","8,684,715"
8,"10,975,017",3.27%,"10,835,491","9,574,586",NC,"1,893,810","2,206,287","2,559,123","3,170,276","3,571,623","4,061,929","4,556,155","5,082,059","5,881,766","6,628,637","8,049,313","8,407,248"
9,"10,041,241",2.99%,"10,037,261","9,877,597",MI,"2,420,982","2,810,173","3,668,412","4,842,325","5,256,106","6,371,766","7,823,194","8,875,083","9,262,078","9,295,297","9,938,444","10,079,985"


In [105]:
population_df["1900"].str.replace(",","")

0     1485053
1     3048710
2      528542
3     7268894
4     6302115
5     4821550
6     4157545
7     2216331
8     1893810
9     2420982
10    1883669
11    1854184
12     518103
13     122931
14    2020616
15    2805346
16    2516462
17    3106665
18    1188044
19    2069042
20     539700
21    1751394
22    1340316
23    1828697
24    1381625
25    2147174
26     413536
27     790391
28     908420
29     276749
30    2231853
31      42335
32    1311564
33    1470495
34    1551270
35     195310
36     161772
37    1066300
38     958800
39     154001
40     411588
41     694466
42     243329
43     428556
44     184735
45     401570
46     319146
47      63592
48     343641
49      92531
Name: 1900, dtype: object

In [111]:
population_df["2024"].astype(int)

0     38889770
1     30976754
2     22975931
3     19469232
4     12951275
5     12516863
6     11812173
7     11145304
8     10975017
9     10041241
10     9320865
11     8752297
12     7841283
13     7497004
14     7204002
15     7020058
16     6892124
17     6215144
18     6196525
19     5931367
20     5914181
21     5761530
22     5464155
23     5143033
24     4559475
25     4540745
26     4227337
27     4088377
28     3625646
29     3454232
30     3214315
31     3210931
32     3089060
33     2944376
34     2940452
35     2115266
36     1990456
37     1988698
38     1766107
39     1430877
40     1405105
41     1402106
42     1142746
43     1098082
44     1044321
45      928767
46      788940
47      733536
48      647818
49      586485
Name: 2024, dtype: int32

In [112]:
years = ["2024","2023","2010",'1900', '1910', '1920','1930', '1940', '1950', '1960', '1970', '1980', '1990', '2000', '2003']
for year in years:
    print(year)
    population_df[year] = population_df[year].str.replace(",","")
    population_df[year] = population_df[year].astype(int)

2024
2023
2010
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2003


In [114]:
population_df["per_2024"] = population_df["per_2024"].str.replace("%","")
population_df["per_2024"] = population_df["per_2024"].astype(float)

In [116]:
population_df["1900"].sum()

75933450

In [117]:
for year in ["2023","2010",'1900', '1910', '1920','1930', '1940', '1950', '1960', '1970', '1980', '1990', '2000', '2003']:
    population_df[f"per_{year}"] = population_df[year]/population_df[year].sum()


In [ ]:
for a in ["State","main_key","Sector"]:
    for c in ["NoEmp","DisbursementGross","GrAppv","SBA_Appv"]

In [88]:
population_df = pd.concat([population_df,population_df_02],axis=1)